Load the data sarcasm.json

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json

--2021-12-29 10:38:35--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.5.128, 74.125.133.128, 74.125.140.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.5.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.04s   

2021-12-29 10:38:36 (120 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import json

In [3]:
with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

In [4]:
training_size = 20000
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [6]:
vocab_size = 10000
tokenizer = Tokenizer(num_words=vocab_size,oov_token='<OOV>')
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

In [8]:
max_length = 100
trunc_type='post'
padding_type='post'

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_pad = pad_sequences(training_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_pad = pad_sequences(testing_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

In [9]:
training_padded = np.array(training_pad)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_pad)
testing_labels = np.array(testing_labels)

In [10]:
embedding_dim = 16

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [12]:
num_epochs = 50
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/50
625/625 - 5s - loss: 0.6564 - accuracy: 0.5938 - val_loss: 0.5622 - val_accuracy: 0.6836 - 5s/epoch - 8ms/step
Epoch 2/50
625/625 - 3s - loss: 0.4235 - accuracy: 0.8337 - val_loss: 0.4042 - val_accuracy: 0.8143 - 3s/epoch - 4ms/step
Epoch 3/50
625/625 - 3s - loss: 0.3089 - accuracy: 0.8803 - val_loss: 0.3527 - val_accuracy: 0.8541 - 3s/epoch - 5ms/step
Epoch 4/50
625/625 - 3s - loss: 0.2553 - accuracy: 0.9014 - val_loss: 0.3468 - val_accuracy: 0.8532 - 3s/epoch - 4ms/step
Epoch 5/50
625/625 - 3s - loss: 0.2215 - accuracy: 0.9155 - val_loss: 0.3456 - val_accuracy: 0.8544 - 3s/epoch - 5ms/step
Epoch 6/50
625/625 - 3s - loss: 0.1918 - accuracy: 0.9272 - val_loss: 0.3557 - val_accuracy: 0.8524 - 3s/epoch - 5ms/step
Epoch 7/50
625/625 - 3s - loss: 0.1718 - accuracy: 0.9363 - val_loss: 0.3638 - val_accuracy: 0.8535 - 3s/epoch - 4ms/step
Epoch 8/50
625/625 - 3s - loss: 0.1525 - accuracy: 0.9450 - val_loss: 0.3978 - val_accuracy: 0.8436 - 3s/epoch - 4ms/step
Epoch 9/50
625/625 - 3s 

In [13]:
sentance = [
            'granny starts to fear spiders in the garden might be real',
            'the weather today is bright and sunny'
]
sequences = tokenizer.texts_to_sequences(sentance)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

[[9.1870897e-05]
 [3.1856744e-09]]
